In [1]:
import requests
from rdflib import Graph, Namespace, Literal, URIRef

In [15]:
def get_movies_directors_in(sparql_endpoint, director_name):
    # Define the DBpedia SPARQL endpoint
    sparql_endpoint = sparql_endpoint
    director_name = director_name


    # Define the SPARQL query to retrieve movies
    sparql_query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbo: <http://dbpedia.org/ontology/>

    SELECT ?movie ?title ?director
    WHERE {{
        ?movie rdf:type dbo:Film;
            rdfs:label ?title;
            dbo:director ?d.
        ?d rdfs:label ?director.
        FILTER(?director = "{}"@en)
        FILTER(LANG(?title)="en")
    }}
    LIMIT 100
    """.format(director_name)

    # Send the SPARQL query to the DBpedia SPARQL endpoint
    response = requests.get(sparql_endpoint, params={'query': sparql_query, 'format': 'json'})
    data = response.json()

    # Parse the JSON response and create an rdflib graph
    graph = Graph()
    graph.parse(data=response.text, format="json-ld")

    # Iterate over the results and print movie titles
    for result in data['results']['bindings']:
        title = result['title']['value']
        print(title)


In [17]:
sparql_endpoint = "http://dbpedia.org/sparql"
get_movies_directors_in(sparql_endpoint, "Christopher Nolan")

Quay (film)
Batman Begins
Memento (film)
The Prestige (film)
Dunkirk (2017 film)
Following
Interstellar (film)
Tenet (film)
The Dark Knight
The Dark Knight Rises
Doodlebug (film)
Inception
Insomnia (2002 film)
Oppenheimer (film)


In [18]:
# for genre
"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dct: <http://purl.org/dc/terms/>

SELECT ?movie ?title ?subjectText
WHERE {
  ?movie rdf:type dbo:Film;
         rdfs:label ?title.

  OPTIONAL {
    ?movie dct:subject ?subject.
?subject rdfs:label ?subjectText.
  }

  FILTER(
    CONTAINS(UCASE(?subjectText), "COMEDY") ||
    CONTAINS(UCASE(?subjectText), "DRAMA") ||
    CONTAINS(UCASE(?subjectText), "HORROR") ||
    CONTAINS(UCASE(?subjectText), "ROMANTIC")
  )



  FILTER(LANG(?title)="en")
}
LIMIT 100"""


#genre count
"""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dct: <http://purl.org/dc/terms/>

SELECT ?movie ?title (COUNT(DISTINCT ?genre) as ?genreCount)
WHERE {
  ?movie rdf:type dbo:Film;
         rdfs:label ?title.

  OPTIONAL {
    ?movie dct:subject ?subject.
    ?subject rdfs:label ?subjectText.
  }

  BIND(
    IF(
      CONTAINS(UCASE(?subjectText), "COMEDY"), "COMEDY",
      IF(
        CONTAINS(UCASE(?subjectText), "DRAMA"), "DRAMA",
        IF(
          CONTAINS(UCASE(?subjectText), "HORROR"), "HORROR",
          IF(CONTAINS(UCASE(?subjectText), "ROMANTIC"), "ROMANTIC", "")
        )
      )
    ) as ?genre
  )

  FILTER(?genre != "")

  FILTER(LANG(?title)="en")
}
GROUP BY ?movie ?title
LIMIT 100
"""

'\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX dbo: <http://dbpedia.org/ontology/>\nPREFIX dct: <http://purl.org/dc/terms/>\n\nSELECT ?movie ?title (COUNT(DISTINCT ?genre) as ?genreCount)\nWHERE {\n  ?movie rdf:type dbo:Film;\n         rdfs:label ?title.\n\n  OPTIONAL {\n    ?movie dct:subject ?subject.\n    ?subject rdfs:label ?subjectText.\n  }\n\n  BIND(\n    IF(\n      CONTAINS(UCASE(?subjectText), "COMEDY"), "COMEDY",\n      IF(\n        CONTAINS(UCASE(?subjectText), "DRAMA"), "DRAMA",\n        IF(\n          CONTAINS(UCASE(?subjectText), "HORROR"), "HORROR",\n          IF(CONTAINS(UCASE(?subjectText), "ROMANTIC"), "ROMANTIC", "")\n        )\n      )\n    ) as ?genre\n  )\n\n  FILTER(?genre != "")\n\n  FILTER(LANG(?title)="en")\n}\nGROUP BY ?movie ?title\nLIMIT 100\n'

In [ ]:
#count variables if made by same director
"""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dct: <http://purl.org/dc/terms/>

SELECT ?movie ?title 
       (COUNT(DISTINCT ?genre) as ?genreCount)
       (COUNT(DISTINCT ?favActor) as ?actorCount)
       (MAX(?isFavoriteDirector) as ?isFavoriteDirector)
WHERE {
  ?movie rdf:type dbo:Film;
         rdfs:label ?title.

  OPTIONAL {
    ?movie dct:subject ?subject.
    ?subject rdfs:label ?subjectText.
  }

  BIND(
    IF(
      CONTAINS(UCASE(?subjectText), "COMEDY"), "COMEDY",
      IF(
        CONTAINS(UCASE(?subjectText), "DRAMA"), "DRAMA",
        IF(
          CONTAINS(UCASE(?subjectText), "HORROR"), "HORROR",
          IF(CONTAINS(UCASE(?subjectText), "ROMANTIC"), "ROMANTIC", "")
        )
      )
    ) as ?genre
  )

  OPTIONAL {
    ?movie dbo:starring ?actorResource.
    ?actorResource rdfs:label ?favActor.
  }

  OPTIONAL {
    ?movie dbo:director ?directorResource.
    ?directorResource rdfs:label ?director.
  }

  BIND(
    IF(CONTAINS(UCASE(?favActor), "FAVORITE_ACTOR"), "FAVORITE_ACTOR", "") as ?actor
  )

  BIND(
    IF(CONTAINS(UCASE(?director), "FAVORITE_DIRECTOR"), 1, 0) as ?isFavoriteDirector
  )

  FILTER(?genre != "" || ?actor != "")

  FILTER(LANG(?title)="en")
}
GROUP BY ?movie ?title
LIMIT 10
"""